# Example of changing Streambank parameters

We're going to change the streambank height parameter and run Source with the original and changed values

In [1]:
# Some steps required until these Python modules are properly installed...
import sys
sys.path.append('../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
import gbr

In [2]:
# Point the system at a particular output directory...
gbr.init('D:/Beckers/outputs/Scenario 1/')

In [3]:
# Initialise the Veneer (Source scripting tool)
v = gbr.veneer()

In [4]:
# Also, lets switch on the performance options
v.configureOptions({'RunNetworksInParallel':True,'PreRunCatchments':True,'ParallelFlowPhase':True})
v.model.sourceScenarioOptions("PerformanceConfiguration","ProcessCatchmentsInParallel",True)


In [5]:
# Its a good idea to set some options in Dynamic Sednet to prevent the results window appearing
# Also, to make it automatically override existing results
v.configureOptions({'ShowResultsAfterRun':False,'OverwriteResults':True})

In [6]:
# Query terms to find the BankHeight_M parameter...
# (Not easy!)
namespace = 'RiverSystem.Constituents.LinkElementConstituentData as LinkElementConstituentData'
accessor = 'scenario.Network.ConstituentsManagement.Elements.OfType[LinkElementConstituentData]().*Data.ProcessingModels.Where(lambda x: x.Constituent.Name=="Sediment - Fine").*Model.BankHeight_M'
#v.model.sourceHelp(accessor,namespace=namespace)

In [7]:
# Now run the query and get the current values
orig_bankheight_m = v.model.get(accessor,namespace=namespace)
orig_bankheight_m

[12.7890768022743,
 11.978887166312,
 9.54718912372855,
 10.5701815970102,
 9.76408136712137,
 2]

In [9]:
# Run with those original values

# First, set the name of the run
v.model.set('scenario.CurrentConfiguration.runName','RUN_ORIGINAL_BANKHEIGHT',literal=True)

True

In [10]:
# Now, lets run the model... When this cell executes in Python, the run window should appear in Source...
v.run_model()

(302, 'runs/1')

In [11]:
# NOTE: The above output (eg runs/1) is a point to retrieving the 'normal' Source results - ie all the time series recorders...
# We don't need that for GBR/Dynamic Sednet, because we can get to the summarised results

In [12]:
# Lets take a quick look at those results...
results_original = gbr.Results('RUN_ORIGINAL_BANKHEIGHT')
results_original.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000e+00,0.000000e+00,0.000000e+00
Atrazine,0.000000e+00,0.000000e+00,0.000000e+00
Flow,4.884436e+08,4.195476e+08,1.777145e+08
N_DIN,9.354795e+04,8.003086e+04,3.283155e+04
N_DON,9.354795e+04,8.003086e+04,3.283155e+04
N_Particulate,9.726372e+05,7.073041e+05,4.915708e+05
P_DOP,8.682243e+04,7.540816e+04,3.158951e+04
P_FRP,8.682243e+04,7.540816e+04,3.158951e+04
P_Particulate,5.715282e+05,4.686279e+05,2.785525e+05


In [13]:
# We can set every 'instance' of BankHeight_M - ie in every link- to a single value, with
#
# v.model.set(accessor,2.0)
#
# or we can pass in a list of values
#
# v.model.set(accessor,[0.2,0.3,0.5,0.4,1.0],fromList=True)
#
# Now... If your list of values is shorter than the number of instances... (ie # links),
# then the list will be 'recycled'... That is, the list will be reused repeatedly until values have been assigned to all
# instances...
#
# ie... Given that the Becker's model has 5 Links, [0.2,0.3] to saying [0.2,0.3,0.2,0.3,0.2]

In [14]:
# Set to a constant, 2
v.model.set(accessor,2,namespace=namespace)

True

In [15]:
# Check that it took effect
v.model.get(accessor,namespace=namespace)

[2, 2, 2, 2, 2, 2]

In [16]:
# Now change the run name
v.model.set('scenario.CurrentConfiguration.runName','RUN_CHANGED_BANKHEIGHT',literal=True)

True

In [17]:
v.run_model()

(302, 'runs/2')

In [18]:
results_changed = gbr.Results('RUN_CHANGED_BANKHEIGHT')
results_changed.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000e+00,0.000000e+00,0.000000e+00
Atrazine,0.000000e+00,0.000000e+00,0.000000e+00
Flow,4.884436e+08,4.195476e+08,1.777145e+08
N_DIN,9.354795e+04,8.003086e+04,3.283155e+04
N_DON,9.354795e+04,8.003086e+04,3.283155e+04
N_Particulate,8.526118e+05,8.560010e+05,9.335531e+05
P_DOP,8.682243e+04,7.540816e+04,3.158951e+04
P_FRP,8.682243e+04,7.540816e+04,3.158951e+04
P_Particulate,7.398785e+05,7.645351e+05,8.506552e+05


In [19]:
# Now that we've done both runs, we probably want to put the parameter back to normal...
v.model.set(accessor,orig_bankheight_m,namespace=namespace,fromList=True)

True

In [20]:
v.model.get(accessor,namespace=namespace)

[12.7890768022743,
 11.978887166312,
 9.54718912372855,
 10.5701815970102,
 9.76408136712137,
 2]

In [21]:
# Now... Lets run a results comparison...
differences = gbr.DifferenceResults('RUN_ORIGINAL_BANKHEIGHT','RUN_CHANGED_BANKHEIGHT')
differences.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000,0.000000,0.000000e+00
Atrazine,0.000000,0.000000,0.000000e+00
Flow,0.000000,0.000000,0.000000e+00
N_DIN,0.000000,0.000000,0.000000e+00
N_DON,0.000000,0.000000,0.000000e+00
N_Particulate,120025.364195,-148696.878550,-4.419823e+05
P_DOP,0.000000,0.000000,0.000000e+00
P_FRP,0.000000,0.000000,0.000000e+00
P_Particulate,-168350.304703,-295907.205772,-5.721027e+05
